<a href="https://colab.research.google.com/github/Aeagon07/Pytourch/blob/main/Optuna_Intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
!pip install optuna

In [38]:
# Import Neccessary Libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: 'Sciket-learns' build in 'load_diabetes' is regression model
# we will loaad the actual diabeties dataset  from external sources

import pandas as pd

# Import the datasets
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


### Replace Missing Number

In [39]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


## Train-Test Split

In [40]:
X = df.drop('Outcome', axis=1)
Y = df['Outcome']

# Split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Scalling not required but okay
scale = StandardScaler()
X_train = scale.fit_transform(x_train)
X_test = scale.transform(x_test)

# Check
print(f"Training set shape: {x_train.shape}")
print(f"Testing set shape: {x_test.shape}")

Training set shape: (537, 8)
Testing set shape: (231, 8)


In [41]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 250)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators = n_estimators,
        max_depth = max_depth,
        random_state = 42
    )

    # Perform k-fold cross validation and calculate accurancy(k=3)
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score # return the accurancy score of the optuna

In [42]:
# Create new study object and optimize the objective fucntion
study1 = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler()) # we aim to maximize so directionis maximize
# If in our objective function we are taking of the loss then you have to use direction as the minimize
study1.optimize(objective, n_trials=50)

[I 2025-10-13 20:08:10,381] A new study created in memory with name: no-name-854cd4f2-351c-4553-ad60-ec0cbb2a87c3
[I 2025-10-13 20:08:10,732] Trial 0 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 55, 'max_depth': 8}. Best is trial 0 with value: 0.7746741154562384.
[I 2025-10-13 20:08:11,891] Trial 1 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 223, 'max_depth': 17}. Best is trial 0 with value: 0.7746741154562384.
[I 2025-10-13 20:08:13,348] Trial 2 finished with value: 0.7783985102420856 and parameters: {'n_estimators': 109, 'max_depth': 16}. Best is trial 2 with value: 0.7783985102420856.
[I 2025-10-13 20:08:14,071] Trial 3 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 59, 'max_depth': 7}. Best is trial 2 with value: 0.7783985102420856.
[I 2025-10-13 20:08:14,565] Trial 4 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 53, 'max_depth': 19}. Best is trial 2 with value: 0.778398510

In [43]:
# print out the result
print(f'Best trial Accurancy: {study1.best_trial.value}')
print(f'Best trial hyperparameters: {study1.best_trial.params}')

Best trial Accurancy: 0.7821229050279329
Best trial hyperparameters: {'n_estimators': 119, 'max_depth': 19}


In [44]:
# Now we make the complete new model using these hyperparmeters cause they are the best ones
from sklearn.metrics import accuracy_score

# We test our model on the testing data set

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study1.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


## Execution of the Random Search in Optuna

In [45]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 250)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators = n_estimators,
        max_depth = max_depth,
        random_state = 42
    )

    # Perform k-fold cross validation and calculate accurancy(k=3)
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score # return the accurancy score of the optuna

In [46]:
# Create new study object and optimize the objective fucntion
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())
# Everything is same you just update the smapler only
study.optimize(objective, n_trials=50)

[I 2025-10-13 20:08:46,226] A new study created in memory with name: no-name-97aadd6d-9e13-4188-9a98-d6826b26bd70
[I 2025-10-13 20:08:46,504] Trial 0 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 60, 'max_depth': 6}. Best is trial 0 with value: 0.7635009310986964.
[I 2025-10-13 20:08:47,727] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 233, 'max_depth': 7}. Best is trial 1 with value: 0.7672253258845437.
[I 2025-10-13 20:08:49,020] Trial 2 finished with value: 0.7560521415270017 and parameters: {'n_estimators': 220, 'max_depth': 4}. Best is trial 1 with value: 0.7672253258845437.
[I 2025-10-13 20:08:49,975] Trial 3 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 247, 'max_depth': 18}. Best is trial 3 with value: 0.7746741154562384.
[I 2025-10-13 20:08:50,779] Trial 4 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 158, 'max_depth': 12}. Best is trial 3 with value: 0.77467411

In [47]:
# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')
# got the new trial parameter for the random search they are not the best one

Best trial accuracy: 0.7802607076350093
Best hyperparameters: {'n_estimators': 121, 'max_depth': 19}


In [48]:
# Run the model on testing dataset using the hyperparamters

from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)
print(study.best_trial.params)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with Random Search hyperparameters: {test_accuracy:.2f}')

{'n_estimators': 121, 'max_depth': 19}
Test Accuracy with Random Search hyperparameters: 0.74


## Execution of the Grid Search

In [49]:
# for the grid search technique you have to define the search interval first then you just pass the search sapce into the samplers
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [50]:
# Create new study object and optimize the objective fucntion
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective) # no need of the number of trials cause we define the search space already here it goes on upto 16 (4*4)

[I 2025-10-13 20:09:27,338] A new study created in memory with name: no-name-f1da0c2b-d47e-44bc-a3f1-3e332de3ce8f
[I 2025-10-13 20:09:27,726] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-10-13 20:09:28,551] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-10-13 20:09:29,213] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-10-13 20:09:30,369] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-10-13 20:09:31,158] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [51]:
# print the output
print(study.best_trial.value)
print(study.best_trial.params)

0.7746741154562384
{'n_estimators': 50, 'max_depth': 5}


In [52]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the Grid Search hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)
print(study.best_trial.params)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with Grid Search hyperparameters: {test_accuracy:.2f}')


{'n_estimators': 50, 'max_depth': 5}
Test Accuracy with Grid Search hyperparameters: 0.74


# Optuna Visualization

In [53]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [54]:
# 1. Optimization History
plot_optimization_history(study1).show()
# graph between the trial number and the objective function -> you can take some info to plan our trail cause if you see after the 31st trail graph is not increaseing

In [55]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study1).show()
#  It helps you to understand between the max_depth, n_estiometers and the objective function

#  you have to where the number of line are more, where the number of trials are more it is hotspot where number trails conducted more that means it is promissing region

In [56]:
# 3. Slice Plot
plot_slice(study1).show()
# We plot the evry induvuial feature or parameter with the objective function

In [57]:
# 4. Contour Plot
plot_contour(study1).show()
# more the dark more the accurancy
#  you clearly seen where he intelligencly get more more trials in darker region as compared to the light region

In [58]:
# Hyperparameter Importance
plot_param_importances(study1).show()
# It shows us which hyperparameter is more important..

# Optimizing on Same Datasets on Multiple Models and find the best models and that's models best HP's

In [59]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [60]:
# Define the Objective Function for Optuna
def objective(trial):
  classifier_name = trial.suggest_categorical('classifier', ['RandomForest', 'GradientBoosting', 'SVM'])

  if classifier_name == 'SVM':
    # SVM Hyperparameter
    c = trial.suggest_float('c', 0.1, 100, log=True)
    kernal = trial.suggest_categorical('kernal', ['linear', 'rbf', 'poly', 'sigmoid'])
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

    model = SVC(C=c, kernel=kernal, gamma=gamma, random_state=42)

  elif classifier_name == 'RandomForest':
    # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

  elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )
  # Perform cross-validation and return the mean accuracy
  score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
  return score

In [61]:
# create the study model
study = optuna.create_study(direction = 'maximize')
study.optimize(objective, n_trials = 100)

[I 2025-10-13 20:12:00,219] A new study created in memory with name: no-name-5961701a-3dfa-472a-8c5a-5d878d2e0065
[I 2025-10-13 20:12:00,593] Trial 0 finished with value: 0.7635009310986964 and parameters: {'classifier': 'RandomForest', 'n_estimators': 75, 'max_depth': 14, 'min_samples_split': 6, 'min_samples_leaf': 2, 'bootstrap': False}. Best is trial 0 with value: 0.7635009310986964.
[I 2025-10-13 20:12:01,700] Trial 1 finished with value: 0.7821229050279329 and parameters: {'classifier': 'RandomForest', 'n_estimators': 277, 'max_depth': 5, 'min_samples_split': 10, 'min_samples_leaf': 1, 'bootstrap': False}. Best is trial 1 with value: 0.7821229050279329.
[I 2025-10-13 20:12:03,922] Trial 2 finished with value: 0.7430167597765364 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 214, 'learning_rate': 0.03183705316754097, 'max_depth': 16, 'min_samples_split': 7, 'min_samples_leaf': 7}. Best is trial 1 with value: 0.7821229050279329.
[I 2025-10-13 20:12:05,168] Trial 

In [62]:
# Retrive the best parameter
best_trial = study.best_trial
print("Best trial Parameter:", best_trial.params)
print("Best trial Accurancy:", best_trial.values)

Best trial Parameter: {'classifier': 'SVM', 'c': 0.11680397079079038, 'kernal': 'linear', 'gamma': 'auto'}
Best trial Accurancy: [0.7895716945996275]


In [63]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_bootstrap,params_c,params_classifier,params_gamma,params_kernal,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.763501,2025-10-13 20:12:00.220463,2025-10-13 20:12:00.593737,0 days 00:00:00.373274,False,NaN,RandomForest,NaN,NaN,NaN,14.0,2.0,6.0,75.0,COMPLETE
1,1,0.782123,2025-10-13 20:12:00.594937,2025-10-13 20:12:01.700584,0 days 00:00:01.105647,False,NaN,RandomForest,NaN,NaN,NaN,5.0,1.0,10.0,277.0,COMPLETE
2,2,0.743017,2025-10-13 20:12:01.701503,2025-10-13 20:12:03.922861,0 days 00:00:02.221358,NaN,NaN,GradientBoosting,NaN,NaN,0.031837,16.0,7.0,7.0,214.0,COMPLETE
3,3,0.761639,2025-10-13 20:12:03.923723,2025-10-13 20:12:05.168562,0 days 00:00:01.244839,NaN,NaN,GradientBoosting,NaN,NaN,0.062666,18.0,8.0,5.0,132.0,COMPLETE
4,4,0.744879,2025-10-13 20:12:05.169377,2025-10-13 20:12:05.194985,0 days 00:00:00.025608,NaN,0.619725,SVM,scale,sigmoid,NaN,NaN,NaN,NaN,NaN,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.709497,2025-10-13 20:12:28.011289,2025-10-13 20:12:28.038115,0 days 00:00:00.026826,NaN,0.116276,SVM,scale,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.785847,2025-10-13 20:12:28.038807,2025-10-13 20:12:28.065377,0 days 00:00:00.026570,NaN,0.255328,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.787709,2025-10-13 20:12:28.066268,2025-10-13 20:12:28.086276,0 days 00:00:00.020008,NaN,0.100641,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.785847,2025-10-13 20:12:28.086975,2025-10-13 20:12:28.111673,0 days 00:00:00.024698,NaN,1.830461,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE


In [64]:
study.trials_dataframe()['params_classifier'].value_counts()

,count
params_classifier,
SVM,78
RandomForest,11
GradientBoosting,11


In [66]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

,value
params_classifier,
GradientBoosting,0.746403
RandomForest,0.762147
SVM,0.776584


In [67]:
# 1) Optimization History
plot_optimization_history(study).show()

In [68]:
# 3. Slice Plot
plot_slice(study).show()

In [69]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()